<a href="https://colab.research.google.com/github/jopapo/nuinvest_options_irpf/blob/main/A%C3%A7%C3%B5es_NuInvest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Isso vai instalar a biblioteca de processamento das notas de negociação em PDF e instalar os módulos necessários ao algoritmo

In [ ]:
# Not installed libs

!pip install pdfminer

In [ ]:
# Requirements

import requests
from pathlib import Path
from getpass import getpass
import os
import re

Isso aqui vai setar a autenticação.

Observação: Ele não autentica. Você deve autenticar na sua conta nuinvest e obter o bearer token usando o dev tools do seu navegador.

In [ ]:
# Variables
year = 2021

# You should login to your nuinvest account and get the bearer token from the request
access_token = getpass('bearer token')

session = requests.Session()
session.headers.update({'Authorization': f"Bearer {access_token}"})


Este resumo é interessante mas faltam informações como CNPJ da empresa e taxa. É bom pra fazer uma prova real.

In [54]:
# Get Invoices Pdfs

def get_summary():

  response = session.get(f"https://www.nuinvest.com.br/api/gringott/tradingSummary/1?startDate={year}-01-01&endDate={year}-12-31")
  response.raise_for_status()

  print(response)

  trades = []
  if response.status_code == 204:
    print(f'No data period')
  else:
    trades = response.json()['value']['statements']
  
  totals = {}
  for trade in trades:
    total = totals.get(trade['ticker'])
    if not total:
      total = {
          'sellQuantity': 0,
          'buyQuantity': 0,
          'totalBuyValue': 0.0
      }
    total['sellQuantity'] = total['sellQuantity'] + trade['sellQuantity']
    total['buyQuantity'] = total['buyQuantity'] + trade['buyQuantity']
    total['totalBuyValue'] = total['totalBuyValue'] + trade['buyValue']
    totals[trade['ticker']] = total
    #print(trade)

  return totals

summary = get_summary()

for ticker, values in summary.items():
  average_price = values['totalBuyValue'] / values['buyQuantity']
  print(ticker, 'averageBuyValue:', round(average_price, 2), 
        'ownedQuantity:', values['buyQuantity'] - values['sellQuantity'],
        'ownedValue:', round(values['buyQuantity'] * average_price, 2))


# Ações: você pode utilizar o preço médio de compra e o código 31 - Ações.
# FIIs: você pode utilizar o preço médio de compra e o código 73 - Fundos de investimento imobiliário.
# ETFs: você pode utilizar o preço médio de compra e o código 74 - Fundos de investimento de índice de mercado.
# Juros e dividendos creditados e não pagos: você pode utilizar o código 99 - Outros bens e direitos.

<Response [200]>
ABEV3F averageBuyValue: 16.59 ownedQuantity: 17 ownedValue: 281.97
ITUB4F averageBuyValue: 29.12 ownedQuantity: 5 ownedValue: 145.6
MGLU3F averageBuyValue: 7.44 ownedQuantity: 80 ownedValue: 595.4
GTWR11 averageBuyValue: 100.42 ownedQuantity: 3 ownedValue: 4317.85
LVBI11 averageBuyValue: 102.57 ownedQuantity: 52 ownedValue: 5333.8


Esse trecho vai baixar todas as notas de negociação (pdf) da NuInvest localmente.

Observação: Não testado com FI e RV.

In [ ]:
# Get Invoices Pdfs

def get_invoices(prefix):

  response = session.get(f"https://www.nuinvest.com.br/api/gringott/invoices/1/{prefix}?startDate={year}-01-01&endDate={year}-12-31")
  response.raise_for_status()

  print(prefix, response)

  invoices = []
  if response.status_code == 204:
    print(f'No data for {prefix}')
  else:
    invoices = response.json()['value']['invoices']
  
  for invoice in invoices:
    params_values = {key:val for (key,val) in invoice.items() if key in ['invoiceNumber', 'custodyId', 'date']}
    # FI não tem custodyId
    # RV tem o data e não tem custodyid
    # Não testado com TP e PS pq eu não tinha papéis pra isso

    response = session.get(f"https://www.nuinvest.com.br/api/gringott/invoices/report/1/{prefix}", params=params_values, stream=True)
    response.raise_for_status()

    filename = Path(f"Invoice_{prefix}_" + '_'.join(str(x) for x in params_values.values()) + '.pdf')
    filename.write_bytes(response.content)

papers = ['TD', 'TP', 'FI', 'RV', 'PS']
for paper in papers:
  get_invoices(paper)



TD <Response [200]>
TP <Response [204]>
No data for TP
FI <Response [200]>
RV <Response [200]>
PS <Response [204]>
No data for PS


Esse trecho minera os PDFs e transforma em texto para ser mais fácil interpretar os dados.

In [ ]:
# Process all pdfs to txt
%%bash
for f in *.pdf
do
 pdf2txt.py -o $f.txt $f
done

Aqui lemos os textos e sumarizamos o que deve ser declarado e onde no IRPF.

Importante: Não nos responsabilizamos pela corretude dessas informações. Use por sua conta e risco.

In [51]:
# FI = fundo de investimento
# TD = tesouro direto
# TP = tesouro privado
# RV = B3 ações/opções
# PS = títulos públicos

data = {}

for entry in os.scandir('.'):
    if entry.is_file() and entry.name.endswith('.txt'):
      txt = Path(entry).read_text()
      #print(entry)

      if entry.name.startswith('Invoice_FI_'):
        matches = re.search(r"\n\bCNPJ Fundo\n(.+)\b", txt)
        cnpj = matches.group(1)

        cumulated = data.get(cnpj)
        if not cumulated:
          cumulated = {}

          matches = re.search(r"\n\bFundo\n(.+)\b", txt)
          cumulated['name'] = matches.group(1)
          
          cumulated['buy_quantity'] = 0
          cumulated['total_buy_value'] = 0
          cumulated['sell_quantity'] = 0
          cumulated['irpf_code'] = 73 # FIIs: você pode utilizar o preço médio de compra e o código 73 - Fundos de investimento imobiliário.

        matches = re.search(r"\n\bQuantidade de cotas\n(.+)\b", txt)
        share_qty = float(matches.group(1).replace('.','').replace(',','.'))

        #matches = re.search(r"\n\bValor da Cota\n(.+)\b", txt)
        #share_value = matches.group(1)
        
        matches = re.search(r"\n\bValor da Operação\n(.+)\b", txt)
        operation_value = float(matches.group(1).replace('.','').replace(',','.').replace('R$ ',''))

        matches = re.search(r"\n\bNota de\n(.+)\b", txt)
        trade_type = matches.group(1)
        if trade_type == 'APLICAÇÃO':
          cumulated['buy_quantity'] = cumulated['buy_quantity'] + share_qty
          cumulated['total_buy_value'] = cumulated['total_buy_value'] + operation_value
        else:
          cumulated['sell_quantity'] = cumulated['sell_quantity'] + share_qty

        cumulated['average_buy_value'] = cumulated['total_buy_value'] / cumulated['buy_quantity']
        cumulated['owned_total'] = cumulated['average_buy_value'] * (cumulated['buy_quantity'] - cumulated['sell_quantity'])

        data[cnpj] = cumulated

      elif entry.name.startswith('Invoice_TD_'):
        matches = re.search(r"\n\bTipo\n(.+)\b", txt)
        
        cnpj = '62.169.875/0001-79' # Tesouro Direto é a própria corretora

        cumulated = data.get(cnpj)
        if not cumulated:
          cumulated = {}

          matches = re.search(r"\n\bTítulo\n(.+)\b", txt)
          cumulated['name'] = matches.group(1)
          
          cumulated['buy_quantity'] = 0
          cumulated['total_buy_value'] = 0
          cumulated['sell_quantity'] = 0
          cumulated['irpf_code'] = 45 # 45 – Aplicação de renda fixa (CDB, RDB e outros)

        matches = re.search(r"\n\bQuantidade\n(.+)\b", txt)
        share_qty = float(matches.group(1).replace('.','').replace(',','.'))

        #matches = re.search(r"\n\bValor 1 título\n(.+)\b", txt)
        #share_value = matches.group(1)
        
        matches = re.search(r"\n\bValor Total\n(.+)\b", txt)
        operation_value = float(matches.group(1).replace('.','').replace(',','.').replace('R$ ',''))

        matches = re.search(r"\n\bTipo\n(.+)\b", txt)
        trade_type = matches.group(1)
        if trade_type == 'Compra':
          cumulated['buy_quantity'] = cumulated['buy_quantity'] + share_qty
          cumulated['total_buy_value'] = cumulated['total_buy_value'] + operation_value
        else:
          cumulated['sell_quantity'] = cumulated['sell_quantity'] + share_qty

        cumulated['average_buy_value'] = cumulated['total_buy_value'] / cumulated['buy_quantity']
        cumulated['owned_total'] = cumulated['average_buy_value'] * (cumulated['buy_quantity'] - cumulated['sell_quantity'])

        data[cnpj] = cumulated

      elif entry.name.startswith('Invoice_RV_'):

        # Isso vai ser pego pelo bloco anterior.
        pass

        # regex = r"(\bFRACIONARIO\b|\bVISTA\b)"
        # matches = re.finditer(regex, txt, re.MULTILINE)
        # for matchNum, match in enumerate(matches, start=1):
        #   print(match)
        #   data[matchNum] = {'type': match.group(0)}
        #   word = re.compile("\w+", re.MULTILINE)
        #   next_word = word.match(txt, match.endpos)
        #   print(next_word)
        #   data[matchNum]['option'] = next_word.group(0)

      else:
        print('Não implementado:', entry.name)

      
import json
print(json.dumps(data, indent=4, sort_keys=True))

{
    "09.625.909/0001-00": {
        "average_buy_value": 3.5236081747709656,
        "buy_quantity": 85.14,
        "irpf_code": 73,
        "name": "Artesanal FIC de FIM",
        "owned_total": 300.0,
        "sell_quantity": 0,
        "total_buy_value": 300.0
    },
    "10.783.480/0001-68": {
        "average_buy_value": 2.9682398337785694,
        "buy_quantity": 33.69,
        "irpf_code": 73,
        "name": "Daycoval Classic FIRF CP",
        "owned_total": 100.0,
        "sell_quantity": 0,
        "total_buy_value": 100.0
    },
    "39.572.994/0001-56": {
        "average_buy_value": 112.35955056179775,
        "buy_quantity": 0.89,
        "irpf_code": 73,
        "name": "Easynvest Top A\u00e7\u00f5es",
        "owned_total": 100.0,
        "sell_quantity": 0,
        "total_buy_value": 100.0
    },
    "62.169.875/0001-79": {
        "average_buy_value": 10765.776223776222,
        "buy_quantity": 1.4300000000000002,
        "irpf_code": 45,
        "name": "Tesouro Se